<a href="https://colab.research.google.com/github/Hasvitha-Pola/prodigy_infotech/blob/main/task_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import torch
from transformers import TextDataset, DataCollatorForLanguageModeling
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from transformers import Trainer,  TrainingArguments

In [2]:
os.environ["WANDB_DISABLED"] = "true"

In [4]:
# Load the dataset
with open('/content/input.txt', 'r') as f:
    text = f.read()


In [32]:
text[:20]

'First Citizen:\nBefor'

In [6]:
# Tokenize the dataset
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
tokens = tokenizer.tokenize(text)
train_dataset = TextDataset(
    tokenizer=tokenizer,
    file_path='/content/input.txt',
    block_size=128
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/data/datasets/language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


In [7]:
# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=16,
    save_steps=10_000,
    save_total_limit=2,

    logging_dir='./logs',
)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [8]:
# Load the model
model = GPT2LMHeadModel.from_pretrained('gpt2')


model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [9]:
# Initialize the trainer
trainer = Trainer(
    model=model,
    args=training_args,

    data_collator=DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False),
    train_dataset=train_dataset

)

In [10]:
# Train the model
trainer.train()

Step,Training Loss


TrainOutput(global_step=447, training_loss=3.574251520553691, metrics={'train_runtime': 256.61, 'train_samples_per_second': 27.836, 'train_steps_per_second': 1.742, 'total_flos': 466602246144000.0, 'train_loss': 3.574251520553691, 'epoch': 3.0})

In [16]:
model.save_pretrained('gpt2_my_fine_tuned')



In [17]:
from transformers import PreTrainedTokenizerFast, GPT2LMHeadModel, GPT2TokenizerFast, GPT2Tokenizer

In [20]:
def load_model(model_path):
    model = GPT2LMHeadModel.from_pretrained(model_path)
    return model


def load_tokenizer(tokenizer_path):
    # Instead of trying to load from 'tokenizer_path', which is actually the model path
    # We now explicitly load the pretrained GPT2Tokenizer.
    # This assumes you used the standard GPT2 tokenizer during training.
    tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
    return tokenizer

def generate_text(model_path, sequence, max_length):

    model = load_model(model_path)
    tokenizer = load_tokenizer('gpt2') # Using 'gpt2' here to load the pretrained tokenizer
    ids = tokenizer.encode(f'{sequence}', return_tensors='pt')
    final_outputs = model.generate(
        ids,
        do_sample=True,
        max_length=max_length,
        pad_token_id=model.config.eos_token_id,
        top_k=50,
        top_p=0.95,
    )
    print(tokenizer.decode(final_outputs[0], skip_special_tokens=True))

In [31]:
model_path = "/content/gpt2_my_fine_tuned"
sequence1 = "Write a dramatic monologue from the perspective of a starving Roman citizen, addressing the corrupt ruling class."
max_len = 200
generate_text(model_path, sequence1, max_len)

Write a dramatic monologue from the perspective of a starving Roman citizen, addressing the corrupt ruling class.
'The people should say, 'Marry, for what reason you doth be discontented with the state?'
'I should wish to be the first to complain,' said he, 'not the last.

PRINCE EDWARD:
I am the first to demand you to stay?

HENRY BOLINGBROKE:
Not before one day nor after the twenty, but two.

EDWARD:
Why, I have nothing else: I have all the things that are in
my power, for they are all in need of your aid.

DUKE VINCENTIO:
In my state of mind you call it such a thing, is not I
would not bear it myself? Do you not think it, then, as a great one?

BRUTUS:
No, sir,
